# The purpose of this notebook is to turn the clean_and_merge notebook into a set of functions to process other BMF files. 

## This will in turn give us better timeline data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
cols = pd.read_csv("big_data.nosync/BMF_2015_cleaned.csv").columns
cols

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['EIN', 'ASSETS', 'CITY', 'FILER', 'FIPS', 'FNDNCD', 'INCOME', 'LEVEL1',
       'LEVEL2', 'LEVEL3', 'LEVEL4', 'MAJGRPB', 'MSA_NECH', 'NAME', 'NTEE1',
       'NTEECC', 'NTEESRC', 'OUTNCCS', 'OUTREAS', 'PMSA', 'RANDNUM', 'STATE',
       'SUBSECCD', 'ZIP5', 'NTEECONF', 'NTEEFINAL', 'NTEEFINAL1', 'NTMAJ10',
       'NTMAJ12', 'NTMAJ5'],
      dtype='object')

Next, changing everything from the last notebook into functions

In [3]:
def open_combine_squish(location, ID):
    #Takes a str location and a str ID (col name)
    #Location includes /
    #Returns a df containing all data from csvs in location, grouped by col name ID (first instance)
    
    #This makes an empty table
    output = pd.DataFrame([])

    #This goes through the individual files and adds them to the table
    for filename in os.listdir(location):
         output = output.append(pd.read_csv(location+filename, low_memory=False))
    
    output = output.groupby(ID).first()
    return output

def only_state(df, state):
    #Takes a dataframe with a column name "STATE"
    #Returns that dataframe where "STATE" is state
    
    return df[df['STATE'] == state]

def overlap_columns(df):
    #Changes all DF colnames to caps, keeps only columns that overlap with BMF 2015 and BMF 1989
    
    df.columns = map(str.upper, df.columns)
    overlap = [x for x in df.columns if x in cols]
    return df[overlap]

In [4]:
#Here are all the BMF files we're working with

single_files = ["bmf.bm9508.csv", "bmf.bm9606.csv", "bmf.bm9710.csv", "bmf.bm9809.csv", "bmf.bm9912.csv",
               "bmf.bm0005.csv", "bmf.bm0107.csv"]
folders = ["2002_BMF"]
years = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002"]

In [5]:
BMFs = {}
for file in single_files:
    tmp = pd.read_csv("big_data.nosync/"+file)
    tmp = only_state(tmp, "CA")
    tmp = overlap_columns(tmp)
    BMFs.update({file: tmp})

/Users/katiawilliams/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (15,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
BMFs.get("bmf.bm9508.csv").head()

,EIN,NAME,CITY,STATE,FNDNCD,SUBSECCD,ASSETS,INCOME,ZIP5,FIPS,...,OUTNCCS,OUTREAS,NTEESRC,NTMAJ10,NTMAJ12,NTMAJ5,FILER,RANDNUM,NTEEFINAL,NTEEFINAL1
416265,581978297,PHI THETA KAPPA SOCIETY,WOODLAND HILLS,CA,15,3,0,0,91367,28049.0,...,IN,NaN,Q,ED,ED,ED,NaN,0.31441,B83,B
908846,237013557,WORLD WIDE FELLOWSHIP OF THE LIVING GOSPEL BRO...,LOS ANGELES,CA,10,3,0,0,90001,6037.0,...,IN,NaN,Q,PU,PU,OT,NaN,0.50810,S82,S
908847,237076031,EVANGELISTIC FELLOWSHIP CENTER CHURCH OF LOS A...,LOS ANGELES,CA,10,3,0,0,90001,6037.0,...,IN,NaN,Q,RE,RE,OT,NaN,0.48290,X21,X
908848,237140276,SAINT JOHN SPIRITUAL CHURCH OF CHRIST INC,LOS ANGELES,CA,10,3,0,0,90001,6037.0,...,IN,NaN,Q,RE,RE,OT,NaN,0.89019,X21,X
908849,237276317,LIVING GOSPEL FELLOWSHIP DELIVERANCE CHURCHES,LOS ANGELES,CA,10,3,0,0,90001,6037.0,...,IN,NaN,Q,PU,PU,OT,NaN,0.00225,X20,X


In [7]:
for folder in folders:
    tmp = open_combine_squish("big_data.nosync/"+folder+"/", "EIN")
    tmp = only_state(tmp, "CA")
    tmp = overlap_columns(tmp)
    BMFs.update({folder: tmp})

In [8]:
i=0
for file_name,file in BMFs.items():
    if "/" in file_name:
        file_name = file_name[:len(file_name)]
    file.to_csv("big_data.nosync/BMF_"+file_name+"_cleaned.csv")
    i+=1